# Rascunho de Análises
Notebook contendo Análises Exploratória dos Dados utilizados no projeto principal

### Importando pacotes e fazendos ajustes:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import palettable
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

import matplotlib.style
import matplotlib as mpl
for m,n in zip(['grid.color','axes.titlecolor','axes.labelcolor','xtick.color','ytick.color','axes.titlesize',
    'figure.figsize','legend.framealpha'],['white','white','white','white','white','20.0','13.0, 8.0','0.8']):
    mpl.rcParams[m] = n

### Apresentando os dados do Suitability:
Suitability é o questionário que visa mapear o perfil do investidos (mais sucetível a tomar risco ou não). As categorias definidas pela Órama são Conservador, Moderado e Arrojado. Abaixo, ilustramos as perguntas e opções de resposta do questionário. Para facilitar a manuseabilidade dos dados, a seguir trabalharemos somente com os IDs das perguntas e respostas.

In [38]:
questions = pd.read_excel('/Users/pedrocerbino/Downloads/questionario.xlsx')
questions

,id,text,order,id.1,text.1,order.1
0,1,Qual o seu principal objetivo ao investir?,1,1,Preservar meu patrimônio. Eu aceito rendimento...,1
1,1,Qual o seu principal objetivo ao investir?,1,2,Algo entre preservar meu patrimônio e valorizá...,2
2,1,Qual o seu principal objetivo ao investir?,1,3,Valorizar meu patrimônio. Eu aceito correr ris...,3
3,2,Durante quanto tempo você planeja deixar seu c...,2,4,Prefiro não deixar nem um ano inteiro. Tenho p...,1
4,2,Durante quanto tempo você planeja deixar seu c...,2,5,"Só um ano é pouco, mas também não deixo passar...",2
...,...,...,...,...,...,...
22,8,Você tem um capital para investir e descobre u...,8,23,"Não posso deixar passar, mas só aplicaria até ...",2
23,8,Você tem um capital para investir e descobre u...,8,24,"Para atingir este ganho, eu aplicaria mais de ...",3
24,9,"Considerando seus investimentos, qual parcela ...",9,25,Até 15%.,1
25,9,"Considerando seus investimentos, qual parcela ...",9,26,De 16% a 40%.,2


In [1316]:
k = 0
for i,m in zip(questions.text,questions['text.1']):
    if i != k and k != 0:
        print('\n')
    if i != k or k == 0:
        print('\033[1m'+i+'\033[0m')
    print(m)
    k = i

Qual o seu principal objetivo ao investir?
Preservar meu patrimônio. Eu aceito rendimentos menores para evitar perdas.
Algo entre preservar meu patrimônio e valorizá-lo de forma mais expressiva.
Valorizar meu patrimônio. Eu aceito correr riscos maiores para ter ganhos mais significativos.


Durante quanto tempo você planeja deixar seu capital aplicado?
Prefiro não deixar nem um ano inteiro. Tenho planos para essa verba, como fazer uma viagem, por exemplo.
Só um ano é pouco, mas também não deixo passar de três.
Acima de três anos, pois quero aumentar o meu patrimônio no longo prazo


Quanto os seus investimentos representam de sua renda mensal e/ou no seu patrimônio total?
Até 20%.
De 21% a 40%.
Acima de 40%.


Com qual dos produtos abaixo você tem mais familiaridade ou fez aplicações nos últimos 12 meses?
Poupança, Tesouro Direto, Fundo DI e Títulos de Renda Fixa de baixo risco de crédito.
Todos os produtos anteriores e Fundos multimercado.
Todos os produtos anteriores, Fundos de ações

### Importando os dados dos fundos e ralizando Data Wrangling:
Como a fonte dos dados é _.json_, existem hierarquia nos dados (dicionários dentro de outros dicionários). Assim, precisamos realizar ajustes para transformar todos os dados de interesse em um formato manuseável, ou seja, em uma única tabela.

In [867]:
funds = pd.read_json('https://s3.amazonaws.com/orama-media/json/fund_detail_full.json?limit=1000&offset=0&serializer=fund_detail_full')
funds['profile_name'] = [i['fund_suitability_profile']['name'] for i in funds.specification]
funds['profile_num'] = [i['fund_risk_profile']['score_range_order'] for i in funds.specification]
funds.operability = [i['minimum_initial_application_amount'] for i in funds.operability]
funds['macro'] = [i['fund_macro_strategy']['name'] for i in funds.specification]
funds.profitabilities = [i['m12'] for i in funds.profitabilities]
funds.fees = [i['has_anticipated_retrieval'] for i in funds.fees]
funds['type'] = [i['fund_type'] for i in funds.specification]
funds.benchmark = [i['name'] for i in funds.benchmark]
# Renomeamos algumas colunas para melhorar a identificação dos dados
funds.rename(inplace=True,columns={'fees':'anticipated_retrieval','profitabilities':'profit_m12',
                                   'operability':'min_init_amount'})
# Eliminamos informações que não nos são de interesse
funds.drop(inplace=True, columns=['target_fund', 'performance_videos', 'is_closed', 'is_active', 
    'strategy_video', 'closed_to_capture_explanation', 'closing_date', 'fund_manager', 'slug',
    'insurance_company_code', 'fund_situation', 'documents', 'initial_date',
    'performance_audios', 'is_simple', 'opening_date', 'full_name', 'quota_date', 'cnpj'])
funds.head(2)

,anticipated_retrieval,description_seo,min_init_amount,id,simple_name,specification,tax_classification,description,benchmark,orama_standard,volatility_12m,profit_m12,net_patrimony_12m,is_closed_to_capture,esg_seal,profile_name,profile_num,macro,type
0,False,Investir em fundos pela Órama é a melhor forma...,25000.00,1013,PIMCO Income Dólar FIC FIM IE,{'fund_suitability_profile': {'score_range_ord...,Longo prazo,{'objective': 'A política de investimento do F...,CDI,True,0.172383,0.450338,0.000000e+00,False,False,Conservador,4,Renda Fixa,Multimercado
1,False,A Empírica Lótus IPCA FIC FIM CP é um fundo qu...,15000.00,114,Empírica Lótus IPCA FIC FIM CP,{'fund_suitability_profile': {'score_range_ord...,Longo prazo,{'objective': 'O FUNDO tem como objetivo o inv...,CDI,True,0.001569,0.088129,1.604175e+08,False,False,Moderado,8,Renda Fixa,Multimercado


In [838]:
funds.profile_num.value_counts(normalize=True)

10    0.186640
7     0.151277
11    0.137525
9     0.129666
3     0.098232
        ...   
6     0.053045
12    0.037328
5     0.021611
2     0.021611
1     0.001965
Name: profile_num, Length: 12, dtype: float64

### Importando os dados de investimentos de renda Fixa:

In [806]:
bonds = pd.read_json('https://minhaconta.orama.com.br/rest-api/bond?limit=1000&offset=0&serializer=bond_detail')
# Eliminamos informações que não nos são de interesse
bonds.drop(inplace=True,columns=['issuer','complement_bond','bond_acceptance_criteria','bond_type_name',
'pre_fixed_rate','post_fixed_rate','expiration_date','issue_date','maximum_investment_quantity',
'is_sold_out','has_coupon','qualified','application_date','application_time_limit',
'application_datetime_limit','unit_price_precision','orama_rate','benchmark','is_b2c',
'has_reservation_operation','bond_rate_classification'])
bonds.head(2)

,id,annual_profitability,grossed_up_annual_profitability,bond_type,application_unit_price,name,is_warranted_by_fgc,minimum_investment_quantity,is_closed_to_capture,is_tax_exempt,negotiate_own_portfolio,ir_tax,total_days,total_working_days,rate_str,maturity_amount_factor
0,899633,0.127,0.127,CDB,1.0,"CDB Banco Máxima 25/09/2030 12,7% ao ano",True,1000,False,False,False,0.15,3600,2476,"12,7% ao ano",3.237226
1,899552,0.120,0.120,CDB,1.0,CDB Banco Máxima 03/04/2030 12% ao ano,True,1000,False,False,False,0.15,3425,2354,12% ao ano,2.882430


In [805]:
bonds.maturity_amount_factor.value_counts()#(normalize=True)

1.104907    2
1.213058    2
1.211336    2
1.123192    1
1.010123    1
           ..
1.032514    1
1.033015    1
3.237226    1
1.274207    1
1.025491    1
Name: maturity_amount_factor, Length: 128, dtype: int64

In [747]:
bonds.bond_type.value_counts(normalize=True)

CDB                0.514851
LCA                0.237624
LC                 0.178218
CDB Liq. Diaria    0.049505
LCI                0.019802
Name: bond_type, dtype: float64

### Adaptando as respostas do Suitability para o formato da tabela:
Como já mencionado anteriormente, os dados possuem hierarquia, de modo que é preciso adaptá-los para facilitar a manuseabilidade.

In [1016]:
answers = {}
cont = 0
for row in clients_profile.answers:
    for m in range(1,10):
        try:
            answers['answer_'+str(m)].append(row[-1]['answer_'+str(m)])
        except:
            try:
                answers['answer_'+str(m)] = [row[-1]['answer_'+str(m)]]
            except:
                print(int(cont/9))
                print(m)
                print(row,'\n')
                print(row[-1],'\n')
                print('answer_'+str(m),'\n')
        cont += 1

In [1017]:
answers['full_name'] = clients_profile.full_name

In [1019]:
clients_answers = clients_profile.merge(pd.DataFrame(answers),on='full_name')
clients_answers.head(2)

,email,full_name,investor_profile,is_qualified_investor,cpf_cnpj,has_investor_profile,birth_location,state,birth_date,net_asset,mensal_earnings,gender,civil_status,answers,age,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
0,233@invalidmailaddresdomain.com,Nome215,Arrojado,False,100.000.002-15,True,Niterói,Rio de Janeiro,1944-11-24,"Acima de R$1.000.000,00","Até R$5.000,00",Masculino,Casado(a),"[{'investor_profile': '2', 'name': 'Arrojado',...",76,c,c,c,c,d,D,None,None,None
1,234@invalidmailaddresdomain.com,Nome216,Moderado,False,100.000.002-16,True,Rio de Janeiro,Rio de Janeiro,1949-06-12,"De R$200.000,01 a R$300.000,00","Até R$5.000,00",Masculino,Casado(a),"[{'investor_profile': '2', 'name': 'Arrojado',...",71,b,b,b,b,b,b,None,None,None


### Eliminando dados de questionários antigos
Ao longo do tempo, a Órama fez diversas mudanças no seu questionionário de Suitability. Como infelizmente não nos foi disponibilizado o conteúdo dos testes antigos, somentes dos mais recentes, eliminamos as observações que somente possuíam Suitability antigo. Importante ressaltar que, dado que os clientes podem responder o Suitability mais de uma vez, muitos clientes antigos também responderam o Suitability mais recente.

In [1020]:
for k in range(1,10):
    clients_filtered = clients_answers[clients_answers['answer_'+str(k)].isin(['0','1','2'])]
clients_filtered.reset_index(inplace=True,drop=True)

In [1232]:
clients_filtered.head(2)

,full_name,investor_profile,is_qualified_investor,state,net_asset,mensal_earnings,gender,civil_status,age,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9
0,Nome225,1,False,Rio de Janeiro,"De R$50.000,01 a R$100.000,00","De R$5.000,01 a R$10.000,00",Masculino,Solteiro(a),Young,1,1,1,1,1,1,1,1,1
1,Nome236,1,False,Rio de Janeiro,"Até R$50.000,00","De R$5.000,01 a R$10.000,00",Masculino,Solteiro(a),MidAge,1,1,2,2,2,2,2,1,0


### Tranformando as variáveis categóricas em dummies:
O parâmetro _drop_first_ elimina a primeira categoria encontrada, de modo a evitar multicolinearidade no modelo (ou seja, evitar colunas que sejam linearmente dependentes)

In [1135]:
dummies = pd.get_dummies(clients_filtered,drop_first=True,columns=['age','gender','civil_status',
            'state','net_asset','mensal_earnings','is_qualified_investor','answer_1','answer_2',
            'answer_3','answer_4','answer_5','answer_6','answer_7','answer_8','answer_9'])
dummies

,full_name,investor_profile,age_MidAge,age_Old,age_Young,gender_Masculino,civil_status_Divorciado(a),civil_status_Separado(a),civil_status_Solteiro(a),civil_status_União estável,civil_status_Viúvo(a),state_Alagoas,state_Amapá,state_Amazonas,state_Bahia,state_Ceará,state_Distrito Federal,state_Espírito Santo,state_Goiás,state_Maranhão,state_Mato Grosso,state_Mato Grosso do Sul,state_Minas Gerais,state_Paraná,state_Paraíba,state_Pará,state_Pernambuco,state_Piauí,state_Rio Grande do Norte,state_Rio Grande do Sul,state_Rio de Janeiro,state_Rondônia,state_Roraima,state_Santa Catarina,state_Sergipe,state_São Paulo,state_Tocantins,"net_asset_Até R$50.000,00","net_asset_De R$100.000,01 a R$200.000,00","net_asset_De R$200.000,01 a R$300.000,00","net_asset_De R$300.000,01 a R$1.000.000,00","net_asset_De R$50.000,01 a R$100.000,00",net_asset_Nenhum,"mensal_earnings_Até R$5.000,00","mensal_earnings_De R$10.000,01 a R$20.000,00","mensal_earnings_De R$20.000,01 a R$30.000,00","mensal_earnings_De R$30.000,01 a R$100.000,00","mensal_earnings_De R$5.000,01 a R$10.000,00",mensal_earnings_Nenhum,is_qualified_investor_True,answer_1_1,answer_1_2,answer_1_un,answer_2_1,answer_2_2,answer_3_1,answer_3_2,answer_4_1,answer_4_2,answer_5_1,answer_5_2,answer_6_1,answer_6_2,answer_7_1,answer_7_2,answer_7_un,answer_8_1,answer_8_2,answer_9_1,answer_9_2
0,Nome225,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0
1,Nome236,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0
2,Nome238,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,1,0,0,1,0,1,0,1,0,0,0
3,Nome247,2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1
4,Nome261,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2666,Nome72698,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1,1,0,0,1,0,1,0,1,0
2667,Nome72760,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0
2668,Nome72806,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0
2669,Nome73281,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0
